# E-comm Dataset

In [ ]:
%%time
import findspark
findspark.init()

In [ ]:
%%time
from pyspark.sql import SparkSession
from datetime import datetime
user = "Heekang Park" # your name
date = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
spark = SparkSession.builder.master("yarn").appName(f"{user}-{date}").getOrCreate()
spark

## Read Dataset

In [ ]:
%%time
df = spark.read.option("header", "true") \
                  .option("inferSchema", "true") \
                  .csv([
                        "s3://ssds2-e-comm/2019-Dec.csv.gz",
                        # "s3://ssds2-e-comm/2019-Nov.csv.gz",
                        # "s3://ssds2-e-comm/2019-Oct.csv.gz",
                        # "s3://ssds2-e-comm/2020-Apr.csv.gz",
                        # "s3://ssds2-e-comm/2020-Feb.csv.gz",
                        # "s3://ssds2-e-comm/2020-Jan.csv.gz",
                        # "s3://ssds2-e-comm/2020-Mar.csv.gz",
                    ])

print((df.count(), len(df.columns)))

## Preprocess and Summarize Data

In [ ]:
%%time
df_smartphone = df.filter(df.category_code == 'electronics.smartphone')
df_smartphone.summary().show()

In [ ]:
%%time
from pyspark.sql.functions import length, col, max, lpad

print(df_smartphone.schema["product_id"].dataType)
print(df_smartphone.schema["category_id"].dataType)
print(df_smartphone.schema["user_id"].dataType)

df_smartphone = df_smartphone.withColumn("product_id", df_smartphone.product_id.cast('string'))
df_smartphone = df_smartphone.withColumn("category_id", df_smartphone.category_id.cast('string'))
df_smartphone = df_smartphone.withColumn("user_id", df_smartphone.user_id.cast('string'))

max_id_len = df_smartphone.withColumn("product_id_len", length(col("product_id")))\
                          .withColumn("category_id_len",length(col("category_id")))\
                          .withColumn("user_id_len",length(col("user_id")))\
                          .groupBy().agg(max("product_id_len"), max("category_id_len"), max("user_id_len")).collect()

len_pid, len_cid, len_uid = max_id_len[0]

df_smartphone = df_smartphone.withColumn("product_id", lpad(df_smartphone.product_id, len_pid, '0'))\
                             .withColumn("category_id", lpad(df_smartphone.category_id, len_cid, '0'))\
                             .withColumn("user_id", lpad(df_smartphone.user_id, len_uid, '0'))

df_smartphone = df_smartphone.cache()
df_smartphone.show()

In [ ]:
%%time
from pyspark.sql.functions import col, when, count

df_smartphone.select(*(count(when(col(c).isNull(), c)).alias(c) for c in df_smartphone.columns)).show()

In [ ]:
%%time
df_smartphone = df_smartphone.na.drop()
print((df_smartphone.count(), len(df_smartphone.columns)))

In [ ]:
%%time
item_by_brand = df_smartphone.groupby("brand").agg(count("product_id").alias('pid_count'))
item_by_brand = item_by_brand.sort(item_by_brand.pid_count.desc())
item_by_brand.head(10)

In [ ]:
%%time
from pyspark.sql.functions import countDistinct

df_smartphone.select(countDistinct("user_id")).show()
df_smartphone.select(countDistinct("user_session")).show()

In [ ]:
%%time
from pyspark.sql.functions import to_timestamp, date_format

df_smartphone = df_smartphone.withColumn("dt", to_timestamp(col("event_time"),"yyyy-MM-dd HH:mm:ss z"))
df_smartphone = df_smartphone.withColumn("date", date_format('dt', 'yyyy-MM-dd'))
df_smartphone = df_smartphone.withColumn("time", date_format('dt', 'HH:mm:ss'))

df_smartphone.head()

In [ ]:
%%time
from pyspark.sql.functions import dayofweek

df_smartphone = df_smartphone.withColumn("weekday", dayofweek('dt'))
df_smartphone.head()

In [ ]:
%%time
from pyspark.sql.functions import udf

def time_cut(h):
    session_labels = ['0-3', '3-6', '6-9', '9-12', '12-15', '15-18', '18-21', '21-24']
    return session_labels[h // 3]

time_cut_udf = udf(time_cut)

df_smartphone = df_smartphone.withColumn("hour", date_format('dt', 'H').cast('int'))
df_smartphone = df_smartphone.withColumn("time_session", time_cut_udf(col("hour")).cast('string'))
df_smartphone.head(5)

## Data Mining

1. Time distribution analysis

Daily distribution of number of events, by hour

In [ ]:
%%time
view_by_hour = df_smartphone.filter(df_smartphone.event_type == 'view').groupby("hour").agg(count("event_type").alias('view'))
cart_by_hour = df_smartphone.filter(df_smartphone.event_type == 'cart').groupby("hour").agg(count("event_type").alias('cart'))
purchase_by_hour = df_smartphone.filter(df_smartphone.event_type == 'purchase').groupby("hour").agg(count("event_type").alias('purchase'))
event_by_hour = view_by_hour.join(cart_by_hour, 'hour').join(purchase_by_hour, 'hour')

event_by_hour_df = event_by_hour.toPandas()
event_by_hour_df

In [ ]:
%%time
import numpy as np
import matplotlib.pyplot as plt

# plot
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10, 4))
ind = np.arange(event_by_hour_df.index.min(), event_by_hour_df.index.max()+1)
width = 0.5
view_bar = ax1.bar(ind, event_by_hour_df.view, width=width, color='#A0E4F1')
ax1.set_xticks(ind)
ax1.set_xticks(ax1.get_xticks()[::2])
ax1.set_xlabel("Hour in a day")
ax1.set_ylabel("Number of views")
ax1.set_title("Distribution of Views")

width = 0.4
cart_bar = ax2.bar(ind, event_by_hour_df.cart, width=width, color='#7EA6F4')
purchase_bar = ax2.bar(ind+width, event_by_hour_df.purchase, width=width, color='#4A4DE7')
ax2.set_xticks(ind+width/2)
ax2.set_xticks(ax2.get_xticks()[::2])
ax2.set_xticklabels(ind[::2])
ax2.set_xlabel("Hour in a day")
ax2.set_ylabel("Number of events")
ax2.set_title("Distribution of Add-to-Cart and Purchase")
ax2.legend((cart_bar, purchase_bar), ('Add to cart', 'Purchase'))
fig.tight_layout()

Daily distribution of number of events, by time session

In [ ]:
%%time
view_by_time_session = df_smartphone.filter(df_smartphone.event_type == 'view').groupby("time_session").agg(count("event_type").alias('view'))
cart_by_time_session = df_smartphone.filter(df_smartphone.event_type == 'cart').groupby("time_session").agg(count("event_type").alias('cart'))
purchase_by_time_session = df_smartphone.filter(df_smartphone.event_type == 'purchase').groupby("time_session").agg(count("event_type").alias('purchase'))
event_by_time_session = view_by_time_session.join(cart_by_time_session, 'time_session').join(purchase_by_time_session, 'time_session')

event_by_time_session_df = event_by_time_session.toPandas()
event_by_time_session_df

In [ ]:
%%time
# plot
session_labels = ['0-3', '3-6', '6-9', '9-12', '12-15', '15-18', '18-21', '21-24']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
ind = np.arange(event_by_time_session_df.index.nunique())
width = 0.5
ax1.bar(ind, event_by_time_session_df.view, width=width, color='#A0E4F1')
ax1.set_xticks(ind)
ax1.set_xticklabels(session_labels)
ax1.set_xlabel("Time session in a day")
ax1.set_ylabel("Number of views")
ax1.set_title("Distribution of Views")

width = 0.4
cart_bar = ax2.bar(ind, event_by_time_session_df.cart, width=width, color='#7EA6F4')
purchase_bar = ax2.bar(ind+width, event_by_time_session_df.purchase, width=width, color='#4A4DE7')
ax2.set_xticks(ind+width/2)
ax2.set_xticklabels(session_labels)
ax2.set_xlabel("Time session in a day")
ax2.set_ylabel("Number of events")
ax2.set_title("Distribution of Add-to-Cart and Purchase")
ax2.legend((cart_bar, purchase_bar), ('Add to cart', 'Purchase'))
fig.tight_layout()

In [ ]:
%%time
# combination showing ratio
event_by_hour_df['view_pct'] = event_by_hour_df.view / sum(event_by_hour_df.view) * 100
event_by_hour_df['cart_pct'] = event_by_hour_df.cart/ sum(event_by_hour_df.cart) * 100
event_by_hour_df['purchase_pct'] = event_by_hour_df.purchase / sum(event_by_hour_df.purchase) * 100

hours = np.arange(0, 24)
view_pct_hr = plt.plot(event_by_hour_df.index, event_by_hour_df.view_pct, marker='.', color='#A0E4F1', label='View')
cart_pct_hr = plt.plot(event_by_hour_df.index, event_by_hour_df.cart_pct, marker='.', color='#7EA6F4', label='Add-to-cart')
purchase_pct_hr = plt.plot(event_by_hour_df.index, event_by_hour_df.purchase_pct, marker='.', color='#4A4DE7', label='Purchase')
plt.xticks(hours)
plt.ylim([0, 10])
plt.xlabel("Hour in a day")
plt.ylabel("Percentage of event")
plt.title("Daily % Distribution of Events by Hour")
plt.legend()

In [ ]:
%%time
# combination showing ratio
event_by_time_session_df['view_pct'] = event_by_time_session_df.view / sum(event_by_time_session_df.view) * 100
event_by_time_session_df['cart_pct'] = event_by_time_session_df.cart/ sum(event_by_time_session_df.cart) * 100
event_by_time_session_df['purchase_pct'] = event_by_time_session_df.purchase / sum(event_by_time_session_df.purchase) * 100

time_sessions = np.arange(0, len(session_labels))
view_pct_ss = plt.plot(event_by_time_session_df.index, event_by_time_session_df.view_pct, marker='.', color='#A0E4F1', label='View')
cart_pct_ss = plt.plot(event_by_time_session_df.index, event_by_time_session_df.cart_pct, marker='.', color='#7EA6F4', label='Add-to-cart')
purchase_pct_ss = plt.plot(event_by_time_session_df.index, event_by_time_session_df.purchase_pct, marker='.', color='#4A4DE7', label='Purchase')
plt.xticks(time_sessions, session_labels)
plt.ylim([0, 30])
plt.xlabel("Time session in a day")
plt.ylabel("Percentage of event")
plt.title("Percent Distribution of Events by Time Session")
plt.legend()

Daily distribution of events, facet by weekday

In [ ]:
%%time
import pandas as pd

weekdays = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
weekday_dict = dict(zip(weekdays, list(range(1,8))))

view_by_ss_wkdy = df_smartphone.filter(df_smartphone.event_type == 'view').groupby(['weekday', 'time_session']).agg(count("event_type").alias('num_view')).toPandas()
view_by_ss_wkdy['weekday'] = view_by_ss_wkdy['weekday'].apply(lambda x: weekdays[x-1])
view_by_ss_wkdy['weekday'] = pd.Categorical(view_by_ss_wkdy.weekday, categories=weekdays, ordered=True)
view_by_ss_wkdy['time_session'] = pd.Categorical(view_by_ss_wkdy.time_session, categories=session_labels, ordered=True)
view_by_ss_wkdy.sort_values(by=['weekday', 'time_session'], inplace=True)

cart_by_ss_wkdy = df_smartphone.filter(df_smartphone.event_type == 'cart').groupby(['weekday', 'time_session']).agg(count("event_type").alias('num_cart')).toPandas()
cart_by_ss_wkdy['weekday'] = cart_by_ss_wkdy['weekday'].apply(lambda x: weekdays[x-1])
cart_by_ss_wkdy['weekday'] = pd.Categorical(cart_by_ss_wkdy.weekday, categories=weekdays, ordered=True)
cart_by_ss_wkdy['time_session'] = pd.Categorical(cart_by_ss_wkdy.time_session, categories=session_labels, ordered=True)
cart_by_ss_wkdy.sort_values(by=['weekday', 'time_session'], inplace=True)

purchase_by_ss_wkdy = df_smartphone.filter(df_smartphone.event_type == 'purchase').groupby(['weekday', 'time_session']).agg(count("event_type").alias('num_purchase')).toPandas()
purchase_by_ss_wkdy['weekday'] = purchase_by_ss_wkdy['weekday'].apply(lambda x: weekdays[x-1])
purchase_by_ss_wkdy['weekday'] = pd.Categorical(purchase_by_ss_wkdy.weekday, categories=weekdays, ordered=True)
purchase_by_ss_wkdy['time_session'] = pd.Categorical(purchase_by_ss_wkdy.time_session, categories=session_labels, ordered=True)
purchase_by_ss_wkdy.sort_values(by=['weekday', 'time_session'], inplace=True)

purchase_by_ss_wkdy.head(8)

In [ ]:
%%time
# events per weekday
view_per_wkdy = view_by_ss_wkdy.groupby("weekday")['num_view'].sum().to_frame()
cart_per_wkdy = cart_by_ss_wkdy.groupby("weekday")['num_cart'].sum().to_frame()
purchase_per_wkdy = purchase_by_ss_wkdy.groupby("weekday")['num_purchase'].sum().to_frame()
event_by_weekday = pd.concat([view_per_wkdy, cart_per_wkdy, purchase_per_wkdy], axis=1)
event_by_weekday

In [ ]:
%%time
# plot events per weekday
fig, ax1 = plt.subplots()

width = 0.2
xticks = np.arange(0, len(weekdays))

view_wkdy_plot = ax1.bar(xticks, event_by_weekday.num_view, width=width, color='#A0E4F1')
ax1.set_ylim([0, 20000])
ax1.set_ylabel("Number of views")

ax2 = ax1.twinx()
cart_wkdy_plot = ax2.bar(xticks+width, event_by_weekday.num_cart, width=width, color='#7EA6F4')
purchase_wkdy_plot = ax2.bar(xticks+width*2, event_by_weekday.num_purchase, width=width, color='#4A4DE7')
ax2.set_ylim([0, 1500])
ax2.set_ylabel("Number of ATC & purchase")

plt.legend((view_wkdy_plot, cart_wkdy_plot, purchase_wkdy_plot), ('View (LHS)', 'Add-to-cart (RHS)', 'Purchase (RHS)'))
plt.xticks(xticks+width*2, weekdays)
fig.autofmt_xdate(rotation=45)
plt.title('Weekly Distribution of Events')

In [ ]:
%%time
# Daily split
view_by_ss_wkdy_pvt = view_by_ss_wkdy.pivot(index='weekday', columns='time_session', values='num_view')
view_by_ss_wkdy_pvt.T.plot.bar(
    subplots=True,
    layout=(3,3),
    figsize=(10, 10),
    color='#A0E4F1',
    sharex=True,
    sharey=True,
    legend=False,
    xlabel='Time session',
    title='Daily Distribution of Views by Time Session in a Week',
    grid=True
)

In [ ]:
%%time
# Daily split
cart_by_ss_wkdy_pvt = cart_by_ss_wkdy.pivot(index='weekday', columns='time_session', values='num_cart')
cart_by_ss_wkdy_pvt.T.plot.bar(
    subplots=True,
    layout=(3,3),
    figsize=(10, 10),
    color='#7EA6F4',
    sharex=True,
    sharey=True,
    legend=False,
    xlabel='Time session',
    title='Daily Distribution of Add-to-Cart by Time Session in a Week',
    grid=True
)

In [ ]:
%%time
# Daily split
purchase_by_ss_wkdy_pvt = purchase_by_ss_wkdy.pivot(index='weekday', columns='time_session', values='num_purchase')
purchase_by_ss_wkdy_pvt.T.plot.bar(
    subplots=True,
    layout=(3,3),
    figsize=(10, 10),
    color='#4A4DE7',
    sharex=True,
    sharey=True,
    legend=False,
    xlabel='Time session',
    title='Daily Distribution of Purchase by Time Session in a Week',
    grid=True
)

Hypo testing with Chi-square test

H0 - shoppers have the time tendency for different events in a day

In [ ]:
def chi2_test(obs, exp, main_cat: int = 0, alpha = 0.05):
    from scipy.stats import chisquare

    chi, pval = chisquare(f_obs=obs, f_exp=exp)
    if pval < alpha:
        print(f"Statistically significant under level of sig={alpha} ({pval}). H0 rejected.")
    else:
        print(f"NOT statistically significant under level of sig={alpha} ({pval}). H0 not rejected.")

    return chi, pval

In [ ]:
%%time
def half_of_day(x):
    return 2 if x > 12 else 1

half_of_day_udf = udf(half_of_day)

df_smartphone = df_smartphone.withColumn("half_of_day", half_of_day_udf("hour"))

def event_retype(x):
    return 2 if x in ['cart', 'purchase'] else 1

event_retype_udf = udf(event_retype)

df_smartphone = df_smartphone.withColumn("event_retype", event_retype_udf("event_type"))

In [ ]:
%%time
data_test_psdf = df_smartphone[["half_of_day", "event_retype"]]
data_test_pd = data_test_psdf.toPandas()

In [ ]:
%%time
chi_crosstab = pd.crosstab(data_test_pd.half_of_day, data_test_pd.event_retype, margins=True)
chi_crosstab

In [ ]:
%%time
obs = chi_crosstab.iloc[0, :]
exp = chi_crosstab.iloc[1, :]

obs = obs*exp.sum()/obs.sum() # for matching counts

pval, chi = chi2_test(obs, exp)

In [ ]:
%%time
data_test_psdf = df_smartphone[["weekday", "event_retype"]]
data_test_pd = data_test_psdf.toPandas()

In [ ]:
%%time
chi_ctab_wk = pd.crosstab(data_test_pd.weekday, data_test_pd.event_retype, margins=True)
chi_ctab_wk

In [ ]:
%%time
pval, chi = chi2_test(chi_ctab_wk.iloc[:-1, 0] * (chi_ctab_wk.iloc[:-1, 1].sum()/chi_ctab_wk.iloc[:-1, 0].sum()), chi_ctab_wk.iloc[:-1, 1])

Conclusion: shoppers have different time preference for viewing vs. ATC & check-outs when shopping on e-comm platform.

## Funnel Analysis

Add-to-cart rate (ATCR) and cart abandonment rate

now we're utilizing PySpark Pandas API

In [ ]:
%%time
import pyspark.pandas as ps

data_test = df_smartphone.to_pandas_on_spark()

user_session_by_time_session = data_test.drop_duplicates(subset=['user_session']).groupby("time_session")['user_session'].count()\
    .to_frame().rename(columns={"user_session": "num_user_session"})
user_session_by_time_session.head()

In [ ]:
%%time
cart_sessions = data_test[data_test.event_type == 'cart']
user_session_cart_by_time_session = cart_sessions.groupby("time_session")['user_session'].count()\
    .to_frame().rename(columns={"user_session": "num_user_session_cart"})
user_session_cart_by_time_session.head()

In [ ]:
%%time
ps.set_option('compute.ops_on_diff_frames', True) # only needed for pyspark pandas

atcr_by_time_session = ps.concat([user_session_by_time_session, user_session_cart_by_time_session], axis=1)
atcr_by_time_session['atcr'] = atcr_by_time_session.num_user_session_cart / atcr_by_time_session.num_user_session * 100
atcr_by_time_session.head()

In [ ]:
%%time
purchase_sessions = data_test[data_test.event_type == 'purchase']
user_session_purchase_by_time_session = purchase_sessions.groupby("time_session")['user_session'].count()\
    .to_frame().rename(columns={"user_session": "num_user_session_purchase"})
user_session_purchase_by_time_session.head()

In [ ]:
%%time
ctcr_by_time_session = ps.concat([user_session_cart_by_time_session, user_session_purchase_by_time_session], axis=1)
ctcr_by_time_session['ctcr'] = ctcr_by_time_session.num_user_session_purchase / ctcr_by_time_session.num_user_session_cart * 100
ctcr_by_time_session.head()

In [ ]:
%%time
ctcr_by_time_session['car'] = 100 - ctcr_by_time_session.ctcr
ctcr_by_time_session

In [ ]:
%%time
# visualization
fig, ax1 = plt.subplots()

avg_atcr = 4.5
ax1.plot(atcr_by_time_session.index, atcr_by_time_session.atcr, marker='.', color='#A0E4F1', label='Add-to-cart rate (LHS)')
ax1.axhline(y=avg_atcr, linestyle='--', color='#A0E4F1', linewidth=1.5, label=f'Baseline: {avg_atcr}%')
ax1.set_ylim([0, 8])
ax1.set_xlabel("Time session in a day")
ax1.set_ylabel("ATCR")
ax1.legend(loc="upper right")

ax2 = ax1.twinx()
ax2.plot(ctcr_by_time_session.index, ctcr_by_time_session.car, marker='.', color='#7EA6F4', label='Cart abandonment rate (RHS)')
ax2.axhline(y=70, linestyle='--', color='#7EA6F4', linewidth=1.5, label='Baseline: 70%')
ax2.set_ylim([0, 90])
ax2.set_ylabel("CAR")
ax2.legend(loc="lower right")

plt.title("ATCR and Cart Abandonment Rate")
plt.xticks(time_sessions, session_labels)

Time interval between each decision to act

In [ ]:
%%time
from pyspark.sql.functions import concat

df_test = df_smartphone.limit(10000).cache()

df_test = df_test.withColumn("product_id", df_test.product_id.cast('string'))
df_test = df_test.withColumn("category_id", df_test.category_id.cast('string'))
df_test = df_test.withColumn("user_id", df_test.user_id.cast('string'))

max_id_len = df_test.withColumn("product_id_len", length(col("product_id")))\
                    .withColumn("category_id_len", length(col("category_id")))\
                    .withColumn("user_id_len", length(col("user_id")))\
                    .groupBy().agg(max("product_id_len"), max("category_id_len"), max("user_id_len")).collect()

len_pid, len_cid, len_uid = max_id_len[0]

df_test = df_test.withColumn("product_id", lpad(df_test.product_id, len_pid, '0'))\
                 .withColumn("category_id", lpad(df_test.category_id, len_cid, '0'))\
                 .withColumn("user_id", lpad(df_test.user_id, len_uid, '0'))

df_test = df_test.withColumn("uid", concat(col("user_id"), col("brand"), col("product_id")))
uid_view = df_test.filter(df_test.event_type == 'view').select("uid").distinct()
uid_cart = df_test.filter(df_test.event_type == 'cart').select("uid").distinct()
uid_purchase = df_test.filter(df_test.event_type == 'purchase').select("uid").distinct()

uid_common = uid_view.intersect(uid_cart).intersect(uid_purchase).collect()
uid_common = [ u.uid for u in uid_common ]
len(uid_common)

Filter userid+productid+brand with all three events (view-cart-purchase, with purchase being the end)

Calculate number of sessions between first view to end purchase. What’s the average user session numbers before a customer place an order?

Calculate the time gap between decisions: a) First view – ATC b) ATC – purchase What’s the average time interval between each decision? May use box plot

Calculate the price change from first view, ATC and final payment. Is there any price changes that possibly affect consumer’s decision to buy?

Brand analysis (simply)

In [ ]:
%%time
data_filtered = df_test.filter(df_test.uid.isin(uid_common))
print((data_filtered.count(), len(data_filtered.columns)))

In [ ]:
%%time
from datetime import datetime
import timedelta

def dt64_to_dt(datetime64):
    return datetime.utcfromtimestamp(datetime64.tolist()/1e9)

header = [
            'uid',
            'time_view_to_cart',
            'cart_duration',
            'time_cart_to_purchase',
            'time_view_to_purchase',
            'num_user_sessions',
            'price_change_view_to_cart',
            'price_change_cart_to_purchase'
            ]

interval_df = pd.DataFrame(columns=header)

for i in uid_common:
    data_selected = data_filtered.filter(data_filtered.uid == i).sort("event_time").toPandas()
    purchase_index = data_selected[data_selected.event_type == 'purchase'].index[-1]
    data_selected = data_selected.loc[:purchase_index, :]

    # time gap
    try:
        time_view = dt64_to_dt(data_selected[data_selected.event_type == 'view'].drop_duplicates(subset=['event_type'], keep='first')\
            .event_time.astype(np.datetime64).values[0])
    except IndexError:
        time_view = np.nan

    try:
        time_cart_first = dt64_to_dt(data_selected[data_selected.event_type == 'cart'].drop_duplicates(subset=['event_type'], keep='first')\
            .event_time.astype(np.datetime64).values[0])
    except IndexError:
        time_cart_first = np.nan

    try:
        time_cart_final = dt64_to_dt(data_selected[data_selected.event_type == 'cart'].drop_duplicates(subset=['event_type'], keep='last')\
            .event_time.astype(np.datetime64).values[0])
    except IndexError:
        time_cart_final = np.nan

    try:
        time_purchase = dt64_to_dt(data_selected[data_selected.event_type == 'purchase'].drop_duplicates(subset=['event_type'], keep='first')\
            .event_time.astype(np.datetime64).values[0])
    except IndexError:
        time_purchase = np.nan

    try:
        cart_duration = timedelta.Timedelta(time_cart_final - time_cart_first).total.seconds
    except (AttributeError, TypeError):
        cart_duration = np.nan

    try:
        view_to_cart = timedelta.Timedelta(time_cart_first - time_view).total.seconds
    except (AttributeError, TypeError):
        view_to_cart = np.nan

    try:
        cart_to_purchase = timedelta.Timedelta(time_purchase - time_cart_final).total.seconds
    except (AttributeError, TypeError):
        cart_to_purchase = np.nan

    try:
        view_to_purchase = timedelta.Timedelta(time_purchase - time_view).total.seconds
    except (AttributeError, TypeError):
        view_to_purchase = np.nan

    # user session gap
    user_session_gap = data_selected.user_session.nunique()

    # price change
    try:
        price_view = data_selected[data_selected.event_type == 'view'].drop_duplicates(subset=['event_type'], keep='first').price.values[0]
    except IndexError:
        price_view = np.nan

    try:
        price_cart = data_selected[data_selected.event_type == 'cart'].drop_duplicates(subset=['event_type'], keep='first').price.values[0]
    except IndexError:
        price_cart = np.nan

    try:
        price_purchase = data_selected[data_selected.event_type == 'purchase'].drop_duplicates(subset=['event_type'], keep='first').price.values[0]
    except IndexError:
        price_purchase = np.nan

    price_change_view_to_cart = price_cart / price_view - 1
    price_change_cart_to_purchase = price_purchase / price_cart - 1

    # combine and append
    record = [
                i,
                view_to_cart,
                cart_duration,
                cart_to_purchase,
                view_to_purchase,
                user_session_gap,
                price_change_view_to_cart,
                price_change_cart_to_purchase
                ]

    interval_df.loc[interval_df.shape[0]] = record

interval_df.head(10)

In [ ]:
%%time
interval_df = interval_df.reset_index()
interval_df.set_index('uid', inplace=True)
interval_df = interval_df.astype(float).dropna(how='any')
interval_df.describe()

In [ ]:
%%time
interval_df_new = interval_df.copy()

cols_to_winsor = ['time_view_to_cart', 'cart_duration', 'time_cart_to_purchase', 'time_view_to_purchase']
time_bound = [300, 120, 300, 900]
for i in range(len(cols_to_winsor)):
    # interval_df_new[col] = mstats.winsorize(interval_df_new[col], limits=[0, 0.15])
    col = cols_to_winsor[i]
    timelim = time_bound[i]
    interval_df_new.loc[interval_df_new[col] > timelim, col] = timelim
    interval_df_new.loc[interval_df_new[col] < 0, col] = 0
interval_df_new.describe()

In [ ]:
%%time
flierprops = dict(marker='+', markerfacecolor='g', markersize=15, linestyle='none', markeredgecolor='r')
fig = plt.figure(figsize=(8, 6))

ax1 = fig.add_subplot(221)
ax1.boxplot(interval_df_new.time_view_to_cart, flierprops=flierprops)
ax1.set_title('Time from view to add-to-cart')

ax2 = fig.add_subplot(222)
ax2.boxplot(interval_df_new.cart_duration, flierprops=flierprops)
ax2.set_title('Time between first to last add-to-cart')

ax3 = fig.add_subplot(223)
ax3.boxplot(interval_df_new.time_cart_to_purchase, flierprops=flierprops)
ax3.set_title('Time from add-to-cart to purchase')

ax4 = fig.add_subplot(224)
ax4.boxplot(interval_df_new.time_view_to_purchase, flierprops=flierprops)
ax4.set_title('Time from view to purchase')

fig.tight_layout()

Done!